# Jacobi to Cartesian Coordinates

In [1]:
# Library imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from jacobi import make_data_jacobi, make_dataset_cart_to_jac, make_dataset_jac_to_cart
from jacobi import CartesianToJacobi, JacobiToCartesian
from jacobi import make_model_cart_to_jac, make_model_jac_to_cart

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
N = 100
num_body = 3
data = make_data_jacobi(N=N, num_body=num_body)
m = data['m']
q = data['q']
qj = data['qj']
vj = data['vj']

**Review one example A Matrix as a Sanity Check**

In [6]:
# Cumulative mass
M = np.cumsum(m, axis=-1)
M_tot = keras.layers.Reshape(target_shape=(1,))(M[:, num_body-1])

# Assemble num_body x num_body square matrix converting from q to r
A_shape = (N, num_body, num_body)
A_ = np.zeros(A_shape)
A_[:, 0, :] = m / M_tot
for i in range(1, num_body):
    A_[:, i, 0:i] = -m[:, 0:i] / M[:, i-1:i]
    A_[:, i, i] = 1.0

In [7]:
A_[0]

array([[ 0.98923701,  0.00788368,  0.00287931],
       [-1.        ,  1.        ,  0.        ],
       [-0.9920935 , -0.00790645,  1.        ]])

**Create a Dataset**

In [8]:
# Create a tensorflow Dataset instance in both directions
N = 1024
num_body = 3
batch_size = 64

ds_c2j = make_dataset_cart_to_jac(N=N, num_body=num_body, batch_size=batch_size)
ds_j2c = make_dataset_jac_to_cart(N=N, num_body=num_body, batch_size=batch_size)

In [9]:
# Example batch - cartesian to jacobi
cart, jac = list(ds_c2j.take(1))[0]

In [10]:
# Unpack tensors
m = cart['m']
q = cart['q']
v = cart['v']
qj = jac['qj']
vj = jac['vj']
mu = jac['mu']

# Review shapes
print(f'Example batch sizes:')
print(f'm  = {m.shape}')
print(f'q  = {q.shape}')
print(f'v  = {v.shape}')
print(f'qj = {qj.shape}')
print(f'vj = {vj.shape}')
print(f'mu = {mu.shape}')

Example batch sizes:
m  = (64, 3)
q  = (64, 3, 3)
v  = (64, 3, 3)
qj = (64, 3, 3)
vj = (64, 3, 3)
mu = (64, 3)


### Check the Conversion Against the Dataset Using Layers

In [11]:
def rms(x):
    return np.sqrt(np.mean(np.square(x)))

In [12]:
# Compute Jacobi coordinates from Cartesian
qj_calc, vj_calc, mu_calc = CartesianToJacobi()([m, q, v])

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

RMS Error qj: 5.893e+00
RMS Error vj: 7.980e-01


In [15]:
qj_calc[0]

<tf.Tensor: id=193, shape=(3, 3), dtype=float32, numpy=
array([[-1.5743623e-10,  6.1048377e-10, -1.1633413e-11],
       [-7.4294132e-01, -1.1764593e+01, -1.0197804e+00],
       [ 1.8762201e+01, -9.6576881e+00, -1.1894956e+00]], dtype=float32)>

In [16]:
qj[0]

<tf.Tensor: id=198, shape=(3, 3), dtype=float32, numpy=
array([[  0.        ,   0.        ,   0.        ],
       [ -0.7429413 , -11.764593  ,  -1.0197804 ],
       [ 19.50494   ,   2.10368   ,  -0.16999479]], dtype=float32)>

In [17]:
err_qj[0]

<tf.Tensor: id=203, shape=(3, 3), dtype=float32, numpy=
array([[-1.5743623e-10,  6.1048377e-10, -1.1633413e-11],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-7.4273872e-01, -1.1761368e+01, -1.0195007e+00]], dtype=float32)>

In [13]:
# Compute Cartesian coordinates from Jacobi
q_calc, v_calc = JacobiToCartesian()([m, qj, vj])

# Error vector
err_q = q_calc - q
err_v = v_calc - v

# RMS Errors
rms_q = rms(err_q)
rms_v = rms(err_v)

# Display RMS errors
print(f'RMS Error q: {rms_q:5.3e}')
print(f'RMS Error v: {rms_v:5.3e}')

RMS Error q: 5.881e+00
RMS Error v: 7.954e-01


***Conversion in Both Directions is Very Accurate***

### Check the Conversion Against the Dataset Using Models

In [ ]:
model_c2j = make_model_cart_to_jac(num_body=3)

In [ ]:
model_c2j.summary()

In [ ]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'qj': keras.losses.MeanSquaredError(),
        'vj': keras.losses.MeanSquaredError(),
        'mu': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'qj': 1.0,
                'vj': 1.0,
                'mu': 1.0}

In [ ]:
# Compile the c2e model
model_c2j.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [ ]:
# Invoke model manually with numpy arrays
qj_calc, vj_calc, mu = model_c2j([m, q, v])

# Error vector
err_qj = qj_calc - qj
err_vj = vj_calc - vj

# RMS Errors
rms_qj = rms(err_qj)
rms_vj = rms(err_vj)

# Display RMS errors
print(f'RMS Error qj: {rms_qj:5.3e}')
print(f'RMS Error vj: {rms_vj:5.3e}')

In [ ]:
print(f'Output shapes:')
print(f'qj: {qj.shape}')
print(f'vj: {vj.shape}')
print(f'mu: {mu.shape}')

In [ ]:
model_c2j.evaluate(ds_c2j)

In [ ]:
import rebound

In [ ]:
m = np.array([1.0, 1.0E-6, 1.0E-3])
a = np.array([1.00000, 5.2029])
e = np.array([0.0167, 0.0484])
inc = np.radians([0.00, 1.3044])
Omega = np.radians([0.00, 100.47])
omega = np.radians([102.94, 14.73]) - Omega
f = np.radians([100.46, 34.40]) - Omega
n_years = 100
sample_freq = 10

In [ ]:
num_body = 3

# Array shapes
space_dims = 3
mass_shape = (num_body)
pos_shape = (num_body, space_dims)

# Initialize Cartesian position and velocity to zero
q = np.zeros(pos_shape, dtype=np.float32)
v = np.zeros(pos_shape, dtype=np.float32)
# Initialize Jacobi position and velocity to zero
qj = np.zeros(pos_shape, dtype=np.float32)
vj = np.zeros(pos_shape, dtype=np.float32)
# Initialize gravitational field strength 
mu = np.zeros(mass_shape)

# Create a simulation
sim = rebound.Simulation()

# Set units
sim.units = ('yr', 'AU', 'Msun')

# Add primary with 1 solar mass at origin with 0 velocity
sim.add(m=m[0])
p0 = sim.particles[0]

# Body with the current center of mass
com = p0
mu[0] = sim.G * com.m

# Add bodies 2 through num_body
for i in range(num_body-1):
    # Add body i+1 with Jacobi coordinates indexed by i and mass i+1
    sim.add(m=m[i+1], a=a[i], e=e[i], inc=inc[i], Omega=Omega[i], omega=omega[i], f=f[i])
    # The body that was just added
    p = sim.particles[i+1]
    # Jacobi coordinates of this body are its coordinates less the cumulative center of mass
    qj[i+1, :] = [p.x -  com.x,  p.y - com.y,   p.z  - com.z]    
    vj[i+1, :] = [p.vx - com.vx, p.vy - com.vy, p.vz - com.vz]    
    # Update the center of mass
    com = com + p
    # Gravitational field strength
    mu[i+1] = sim.G * com.m

# Move to center of mass
sim.move_to_com()

# The first Jacobi coordinate in row 0 is the position and velocity of the COM
# This has been set to zero above, so don't need to change it from initialization.

# Save the Cartesian coordinates in the COM frame
for i in range(num_body):
    pi = sim.particles[i]
    q[i, :] = [pi.x, pi.y, pi.z]
    v[i, :] = [pi.vx, pi.vy, pi.vz]


In [ ]:
qj

In [ ]:
vj

In [ ]:
q

In [ ]:
v

In [ ]:
mu